In [ ]:
from tqdm.autonotebook import tqdm
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
import pandas as pd

In [ ]:
df = pd.read_csv('./Тестовый датасет/covid_data_test.csv', sep=',')
# df = pd.read_csv('./предсказание_пандемий_при_помощи_ИИ/covid_data_train.csv', sep=';')

In [ ]:
df

In [ ]:
# df = df[df.name == "Новосибирск"]

In [ ]:
cities = df["name"]
subjects = df["subject"]
lats = df["lat"]
lons = df["lng"]

In [ ]:
overpass = Overpass()

In [ ]:
radius = 70000

data = []
for c, s, lat, lon in zip(cities, subjects, lats, lons):
    result = overpass.query(
        f"""
        (
          node["aeroway"="aerodrome"](around:{radius},{lat},{lon});
          way["aeroway"="aerodrome"](around:{radius},{lat},{lon});
          relation["aeroway"="aerodrome"](around:{radius},{lat},{lon});
        );
        out body;
        """,
        timeout=9000,
    )
    total_aerodrome = len(result.elements())
    internationals_aerodrome = 0
    for a in result.elements():
        try:
            if a.tags()["aerodrome"] == 'international':
                internationals_aerodrome += 1
        except:
            pass
        
    data.append({
        "name": c,
        "subject": s,
        "total_aerodrome": total_aerodrome,
        "internationals_aerodrome": internationals_aerodrome
    })

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df.to_csv("osm_aerodrome_test.csv", index=False)
# df.to_csv("osm_aerodrome_train.csv", index=False)